In [9]:
# import experiments.model_learning.what_is_learned
# works but not sure how to run

In [1]:
%run -i experiments/model_learning/what_is_learned

Preparing data...
Data prepared.
Preparing model...
Model prepared.
Starting TS interpolation experiment...


TypeError: ts_interpolation() missing 3 required positional arguments: 'experiment_params', 'model_params', and 'loaders'